In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
import dateutil.easter as easter

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
from accelerate import Accelerator
import torch.optim as optim

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import time

In [7]:
from tqdm.notebook import tqdm

In [36]:
import gc

## Global Variables ###

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
device

device(type='cuda', index=0)

In [10]:
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [11]:
BASE_DIR = '/sharedHDD/rohit/timeseries_learning/ubiquant/'
DATA_DIR = BASE_DIR+'data/parquet/'
INPUT_DIR = BASE_DIR+'input/'
WEIGHTS_DIR = BASE_DIR + 'weights/'

In [12]:
input_chunk_length = 30
output_chunk_length = 1
embedding_dim = 50 ## Investment id embedding dim

In [13]:
features = [f'f_{i}' for i in range(300)]

## Load data ##

In [14]:
%%time
train_df = pd.read_parquet(DATA_DIR+'train_low_mem.parquet')

CPU times: user 8.97 s, sys: 15.5 s, total: 24.5 s
Wall time: 4.2 s


In [15]:
investment_ids = train_df.investment_id.unique().tolist()

In [16]:
max(investment_ids)

3773

#### make test and train df #####

In [17]:
fold = 1

In [18]:
with open(BASE_DIR+'input/folds.pickle', 'rb') as f:
    folds = pickle.load(f)

In [19]:
train_indxs = folds[fold]['train']
test_indxs = folds[fold]['test']
test_f_df = train_df[train_df.index.isin(test_indxs)].reset_index(drop=True)
train_f_df = train_df[train_df.index.isin(train_indxs)].reset_index(drop=True)

#### Load Training data #####

In [20]:
loaded = np.load(INPUT_DIR+'fold_train_'+str(fold)+'.npz')

In [21]:
X_train = loaded['X_train']
Y_train = loaded['Y_train']

In [22]:
Y_train = Y_train.reshape(-1,1)

In [23]:
print(X_train.shape,Y_train.shape)

(942773, 30, 302) (942773, 1)


#### Make train and val split ####

In [24]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [25]:
print(X_train.shape,Y_train.shape,X_val.shape,Y_val.shape)

(754218, 30, 302) (754218, 1) (188555, 30, 302) (188555, 1)


In [37]:
gc.collect()

234

## Modeling ##

In [38]:
config = {
    'seq_length' : input_chunk_length,
    'num_epochs' : 200,
    'lr' : 0.00001,
    'input_size' : 351,
    'hidden_size' : 351,
    'num_layers' : 1,
    'num_classes' :1, ## This is  output dimension
    'train_shuffle': True,
    'val_shuffle': True,
    'batch_size' : 4096,
    'best_model_name' : str(fold)+'_lstm',
    'bidirectional' : False,
    'only_last_hidden': True
}

In [46]:
class TSDataset(Dataset):
    
    def __init__(self, val=False):
        """
        Args:
        """
#         self.x=x
#         self.y=y
        self.val = val

    def __len__(self):
        return len(X_val) if self.val else len(X_train)

    def __getitem__(self, idx):
        if self.val:
            sample = [torch.Tensor(X_val[idx]), torch.Tensor(Y_val[idx])]
        else:
            sample = [torch.Tensor(X_train[idx]), torch.Tensor(Y_train[idx])]
        return sample

#### Model ####

In [47]:
num_epochs = config['num_epochs']
lr = config['lr']
input_size = config['input_size']
hidden_size = config['hidden_size']
num_layers = config['num_layers']
num_classes = config['num_classes']
seq_length = config['seq_length']
bidirectional = config['bidirectional']
only_last_hidden = config['only_last_hidden']

#### Loss function ######

In [48]:
def pearson_loss(x, y):
    xd = x - x.mean()
    yd = y - y.mean()
    nom = (xd * yd).sum()
    denom = ((xd ** 2).sum() * (yd ** 2).sum()).sqrt()
    return 1 - nom / denom

In [49]:
class LstmTsModel(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers,seq_length):
        super(LstmTsModel, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        cat_input_dim: int = 3774
        
        self.embedding = nn.Embedding(cat_input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,bidirectional=bidirectional)
        
        if bidirectional:
            m=2
        else:
            m=1
        
        if only_last_hidden:
            input_fc_dim = hidden_size*m
        else:
            input_fc_dim = self.seq_length*hidden_size*m
        
        print(input_fc_dim)
        
        self.fc = nn.Sequential(nn.Linear(input_fc_dim, input_fc_dim//2),
                                nn.BatchNorm1d(num_features=input_fc_dim//2),
                                nn.Dropout(0.2),
                                nn.ReLU(),
                                
#                                 nn.Linear(input_fc_dim//4, input_fc_dim//16),
#                                 nn.BatchNorm1d(num_features=input_fc_dim//16),
#                                 nn.Dropout(0.2),
#                                 nn.ReLU(),
                                
#                                 nn.Linear(input_fc_dim//16, input_fc_dim//32),
#                                 nn.BatchNorm1d(num_features=input_fc_dim//32),
#                                 # nn.Dropout(0.2),
#                                 nn.ReLU(),
                                
                                
                                nn.Linear(input_fc_dim//2, self.num_classes)
                                )

    def forward(self, x):
        # Propagate input through LSTM
        
        investment_ids = x[:,:,0].type(torch.LongTensor).to(device)
        
        embeddings = self.embedding(investment_ids)
        
        x = torch.cat([embeddings,x[:,:,1:]], dim=2)
        
        h_out, (h_n, _) = self.lstm(x)
        if only_last_hidden:
            h_out = h_out[:,-1:,:]
        
#         print(h_out.shape)
        h_out = h_out.flatten(start_dim=1)
#         print(h_out.shape)
        
        out = self.fc(h_out)
        
        return out

In [50]:
def run(model,train_dl,val_dl):
    def evaluate(model,valid_loader):
        model.eval()
        valid_loss = 0
        rec_loss = 0
        with torch.no_grad():
            for i, inputs in enumerate(tqdm(valid_loader)):
                dataX = inputs[0]
                dataY = inputs[1]
                outputs = model(dataX)
                loss = criterion(outputs, dataY)
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)
        return valid_loss
    
    def train_and_evaluate_loop(train_loader,model,optimizer,criterion,epoch,lr_scheduler=None,valid_loader=None, best_loss=99999):
        train_loss = 0
        for i, inputs in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            model.train()
            
            dataX = inputs[0]
            dataY = inputs[1]
            outputs = model(dataX)
            loss = criterion(outputs, dataY)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if lr_scheduler:
                lr_scheduler.step()
        
        train_loss /= len(train_loader)
        if valid_loader:
            valid_loss = evaluate(model,valid_loader) 
            print(f"Epoch:{epoch} |Train Loss:{train_loss}|Valid Loss:{valid_loss}")
            if valid_loss <= best_loss:
                print(f"{g_}Loss Decreased from {best_loss} to {valid_loss}{sr_}")

                best_loss = valid_loss
                torch.save(model.state_dict(), WEIGHTS_DIR+config['best_model_name'])
        else:
            print(f"Epoch:{epoch} |Train Loss:{train_loss}")
            
                    
        return best_loss
    
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")

    
    
    optimizer = optim.Adam(model.parameters(),lr=config['lr'],amsgrad=False)
    criterion = pearson_loss
    
    # lr_scheduler = CosineAnnealingWarmupRestarts(optimizer, **config_lr)
    # lr_scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, **config_lr)
    lr_scheduler = None

    model,train_dl,val_dl,optimizer,lr_scheduler,criterion = accelerator.prepare(model,train_dl,val_dl,optimizer,lr_scheduler,criterion)

    best_loss = 9999999
    start_time = time.time()
    for epoch in tqdm(range(config["num_epochs"])):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,model,optimizer,criterion,epoch,lr_scheduler,valid_loader=val_dl, best_loss=best_loss)
        
        end_time = time.time()
        print(f"{m_}Time taken by epoch {epoch} is {end_time-start_time:.2f}s{sr_}")
        start_time = end_time
        
    return best_loss, model

In [51]:
model = LstmTsModel(num_classes, input_size, hidden_size, num_layers,seq_length)

351


In [52]:
train_dl = DataLoader(TSDataset(), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=0)
val_dl = DataLoader(TSDataset(val=True), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=0)

In [53]:
gc.collect()

147

In [54]:
best_loss, model = run(model,train_dl,val_dl)

cuda is used


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch Started:0


  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

...

  0%|          | 0/47 [00:00<?, ?it/s]

Epoch:198 |Train Loss:0.5067439098615905|Valid Loss:0.8411135825705021
Time taken by epoch 198 is 67.70s
Epoch Started:199


  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

Epoch:199 |Train Loss:0.5054275599685876|Valid Loss:0.8428976865524941
Time taken by epoch 199 is 67.30s
